In [5]:
import pickle
import re


In [6]:
def read_sentences(path):
    Sentences = []
    Sentence_Classes = []
    Temp_Sentence = []
    Temp_Sentence_Classes = []
    with open(path, 'r', encoding="utf-8") as file:
        # Iterate through each line
        for line in file:

            # Split the line by the tab character
            parts = line.split('\t')


            # Check if there are at least two parts
            if len(parts) >= 2:
                word = parts[0]
                class_ = parts[1].strip()  # strip() to remove leading/trailing whitespace

                # Do something with the word and class, for example, print them
                #print("Word:", word)
                #print("Class:", class_)


                Temp_Sentence.append(word)
                Temp_Sentence_Classes.append(class_)
                #print(Temp_Sentence)
            else:
                #print("Line does not contain a tab character")
                Sentences.append(Temp_Sentence)
                Sentence_Classes.append(Temp_Sentence_Classes)
                Temp_Sentence = []
                Temp_Sentence_Classes = []
    return Sentences, Sentence_Classes


In [21]:
path_normal_reading = r'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_normalreading_ner.txt'

path_task_reading = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_taskreading_ner.txt"

path_sentiment = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_sentiment_ner.txt"

normal_reading_sentences, normal_reading_classes = read_sentences(path_normal_reading)
task_reading_sentences, task_reading_classes = read_sentences(path_task_reading)
sentiment_sentences, sentiment_classes = read_sentences(path_sentiment)

#combine
Sentences_labels = [item for sublist in [normal_reading_sentences, task_reading_sentences, sentiment_sentences] for item in sublist]
Sentence_Classes = [item for sublist in [normal_reading_classes, task_reading_classes, sentiment_classes] for item in sublist]


In [22]:
print(Sentences_labels[0])

['Henry', 'Ford', 'advocated', 'long-time', 'associate', 'Harry', 'Bennett', 'to', 'take', 'the', 'spot', '.']


In [23]:
print(Sentence_Classes[0])

['B-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O']


In [24]:
def read_EEG_embeddings_labels(path):
    with open(path, 'rb') as file:
        EEG_word_level_embeddings = pickle.load(file)
        EEG_word_level_labels = pickle.load(file)
    return EEG_word_level_embeddings, EEG_word_level_labels

In [61]:
def get_sentences_labels(EEG_word_level_labels, EEG_word_level_embeddings):
    sentences = []
    current_sentence = []
    sentence_EEG_embeddings = []

    # Iterate through the words
    EEG_embedding_index = 0

    for i in range(len(EEG_word_level_labels)):
        # Check if the word marks the start of a new sentence
        word = EEG_word_level_labels[i]
        if word == "SOS":
            # If it does, append the current sentence to the list of sentences
            if current_sentence:
                sentences.append(current_sentence)
                sentence_length = len(current_sentence)
                sentence_EEG_segment = EEG_word_level_embeddings[EEG_embedding_index:sentence_length]
                sentence_EEG_embeddings.append(sentence_EEG_segment)
                EEG_embedding_index += sentence_length
            # Start a new sentence
            current_sentence = []
        else:
            # Add the word to the current sentence
            current_sentence.append(word)
    # Append the last sentence
    if current_sentence:
        sentences.append(current_sentence)
        sentence_length = len(current_sentence)
        sentence_EEG_segment = EEG_word_level_embeddings[EEG_embedding_index:sentence_length]
        sentence_EEG_embeddings.append(sentence_EEG_segment)

    return sentences, sentence_EEG_embeddings

In [62]:
def combine_sentences(sentences):
    # Combine each internal list into a string
    combined_sentences = [' '.join(sentence) for sentence in sentences]
    # Initialize an empty list to store the list of words for each sentence
    list_of_words = []
    # Iterate over each sentence in the list
    for sentence in combined_sentences:
        # Split the sentence into a list of words including punctuation
        words = re.findall(r'\b\w+\b|[^\w\s]', sentence)
        # Append the list of words to the list of lists
        list_of_words.append(words)
    return list_of_words


In [130]:
def get_named_entities(Sentences_labels, Sentence_Classes):
    List_of_NE = []
    List_of_NE_Labels = []
    Named_Entity = []
    Named_Entity_Label = []

    for i in range(len(Sentences_labels)):
        current_sentence = Sentences_labels[i]
        current_sentence_label = Sentence_Classes[i]

        for j in range(len(current_sentence)):
            current_word = current_sentence[j]
            #print(current_word)
            current_word_label = current_sentence_label[j]
            #print(current_word_label)
            if current_word_label != 'O':
                Named_Entity.append(current_word)
                Named_Entity_Label.append(current_word_label)
            else:
                if Named_Entity:
                    List_of_NE.append(Named_Entity)
                    List_of_NE_Labels.append(Named_Entity_Label)

                    Named_Entity = []
                    Named_Entity_Label = []
    return List_of_NE, List_of_NE_Labels

In [227]:
def is_named_entity_in_sentences(named_entity, sentences, embeddings):

    named_entity_eeg_list = []

    for sentence in sentences:
        for i in range(len(sentence) - len(named_entity) + 1):
            if sentence[i:i+len(named_entity)] == named_entity:
                named_entity_eeg = embeddings[i:i+len(named_entity)]
                named_entity_eeg_list.append(named_entity_eeg)

    return named_entity_eeg_list

In [228]:
def get_EEG_Class_NE(words, embeddings):
    # Check if each named entity is in the list of sentences
    NE_EEG_segment = []
    NE_EEG_Class = []
    NE = []

    for i in range(len(List_of_NE)):
        Class = List_of_NE_Labels[i]
        NE = List_of_NE[i]
        EEG_list = is_named_entity_in_sentences(NE, words, embeddings)
        NE_EEG_segment.append(EEG_list)
        NE_EEG_Class.append(Class)
        NE.append(NE)

    return NE_EEG_segment, NE_EEG_Class, NE



In [229]:
train_path = r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs_Sentence.pkl"
test_path = r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\Test_EEG_Text_Pairs_Sentence.pkl"

EEG_word_level_embeddings, EEG_word_level_labels = read_EEG_embeddings_labels(train_path)
Test_EEG_word_level_embeddings, Test_EEG_word_level_labels = read_EEG_embeddings_labels(test_path)

In [230]:
EEG_word_level_sentences, EEG_sentence_embeddings = get_sentences_labels(EEG_word_level_labels, EEG_word_level_embeddings)
Test_EEG_word_level_sentences, Test_EEG_sentence_embeddings = get_sentences_labels(Test_EEG_word_level_labels, Test_EEG_word_level_embeddings)


In [231]:
# Combine the sentences
list_of_words = combine_sentences(EEG_word_level_sentences)
Test_list_of_words = combine_sentences(Test_EEG_word_level_sentences)

In [226]:
# Get the named entities
List_of_NE, List_of_NE_Labels = get_named_entities(Sentences_labels, Sentence_Classes)

In [185]:
# Get the EEG embeddings for the named entities
NE_EEG_segment, NE_EEG_Class, NE = get_EEG_Class_NE(list_of_words, EEG_sentence_embeddings)

In [222]:
#Test set
Test_NE_EEG_segment, Test_NE_EEG_Class, Test_NE = get_EEG_Class_NE(Test_list_of_words, Test_EEG_sentence_embeddings)

157906